In [80]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import time

# response = requests.get("https://events.cmu.edu/event/12319-voter-registration-assistance")
from selenium import webdriver

url = "https://events.cmu.edu/event/10974-like-totally-transformative-cmu-in-the-1980s"
browser = webdriver.Chrome()
browser.get(url)
time.sleep(0.05)
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())
date = soup.select("h5#lw_cal_this_day")[0].text
print(date)

Aug. 30 - Nov. 27, 2024


In [126]:

from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import time
# Read the HTML file
with open('cmu.html', 'r') as file:
    html = file.read()

# Create a BeautifulSoup object
soup = BeautifulSoup(html, 'html.parser')

# Now you can work with the parsed HTML
# For example, you can find elements using CSS selectors
# or extract specific data from the HTML structure
# Read the HTML file
with open('cmu.html', 'r') as file:
    html = file.read()

# Create a BeautifulSoup object
soup = BeautifulSoup(html, 'html.parser')
c ="lw_cal_event"
# Find elements with the 'lw_events_title' class

events = soup.find_all(class_ = "lw_cal_event")

# Now you can work with the parsed HTML
# For example, you can iterate over the found elements
# Create a list to store the table data
table_data = []

# Add the headers to the table data
table_data.append(["Title", "Date", "Time", "Location", "Summary", "Description", "id"])
browser = webdriver.Chrome()
# Split the events into 3 parts
num_events = len(events)
part_size = num_events // 5

for i in range(5):
    # Calculate the start and end indices for each part
    start_index = i * part_size
    end_index = (i + 1) * part_size if i < 4 else num_events

    # Create a list to store the table data for each part
    part_table_data = []

    # Add the headers to the table data
    part_table_data.append(["Title","Date1", "Date2",  "Time", "Location", "Summary", "Description", "id"])

    
    # Iterate over the events in the current part
    for event in events[start_index:end_index]:
        
        #print(event)
        # Extract the relevant information from each event
        id = event["data-id"]
        #s = "h3:has(+ div div.lw_cal_event_list div.lw_cal_event[data-id |= \""+id+"\"])"

        #print(s)
        date1 = event.parent.parent.find_previous_sibling("h3").text
        
        title = None
        event_time = None
        location = None
        summary = None
        description = None

        # 
        # enter to the subpage
        if event.select(".lw_events_title a[href]"):
            link = event.select(".lw_events_title a[href]")[0]
            # go to the link and get the title
            # subtract http:// from link
            link['href'] = link['href'].replace('https://', '')
            link['href'] = 'https://events.cmu.edu/'+link['href']
            url = link['href']
            #print(link['href'])
            
            browser.get(url)
            time.sleep(0.5)
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser')
            #print(soup.prettify())
            if soup.select("h5#lw_cal_this_day"):
                #print("date found:", soup.select("h5#lw_cal_this_day")[0].text)
                if soup.select("h5#lw_cal_this_day")[0].text != "":
                    date2 = soup.select("h5#lw_cal_this_day")[0].text

            if soup.select(".lw_hidden script"):
                json = soup.select(".lw_hidden script")[0].text
                description = re.search(r'(?<="description": ").*?(?=")', json).group(0)
                #print(description)
            
        if event.select(".lw_events_title a"):
            title = event.select(".lw_events_title a")[0].text.strip()
        if event.select(".lw_events_time"):
            event_time = event.select(".lw_events_time")[0].text.strip()
        if event.select(".lw_events_location"):
            location = event.select(".lw_events_location")[0].text.strip()
        if event.select(".lw_events_summary"):
            summary = event.select(".lw_events_summary")[0].text.strip()

        # Add the event information to the table data for the current part
        part_table_data.append([title, date1, date2, event_time, location, summary, description, id])

    # Save the table data for the current part to a CSV file
    df_part = pd.DataFrame(part_table_data[1:], columns=part_table_data[0])
    df_part.to_csv(f"cmu_events_part{i+1}.csv", index=False)




In [127]:
import pandas as pd

# List of file names
file_names = ['cmu_events_part1.csv', 'cmu_events_part2.csv', 'cmu_events_part3.csv', 'cmu_events_part4.csv', 'cmu_events_part5.csv']

# Create an empty DataFrame to store the concatenated data
concatenated_df = pd.DataFrame()

# Iterate over the file names
for file_name in file_names:
    # Read each CSV file into a DataFrame
    df = pd.read_csv(file_name)
    
    # Concatenate the DataFrame to the existing concatenated_df
    concatenated_df = pd.concat([concatenated_df, df], ignore_index=True)

# Save the concatenated DataFrame to a new CSV file
concatenated_df.to_csv('cmu_events.csv', index=False)


In [128]:
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

max_page = 24
table_data = []
table_data.append(["Title", "Time", "Venue", "Location", "Region", "Description", "Price"])
for i in range(max_page):
    link = "https://www.pghcitypaper.com/pittsburgh/EventSearch?page="+str(i+1)+"&v=d"
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = None
    tag = None
    time = None 
    venue = None
    location = None
    region = None
    description = None
    price = None
    blocks = soup.select(".fdn-pres-content")
    for block in blocks:
        if block.select(".fdn-teaser-headline"):
            title = block.select(".fdn-teaser-headline")[0].text.strip()
            print(title)
        if block.select(".fdn-teaser-subheadline"):
            time = block.select(".fdn-teaser-subheadline")[0].text.strip()
            print(time)
        if block.select(".fdn-event-teaser-location"):
            venue = block.select(".fdn-event-teaser-location")[0].text.strip()
            print(venue)
        if block.select(".fdn-inline-split-list.fdn-teaser-infoline span"):
            location = block.select(".fdn-inline-split-list.fdn-teaser-infoline span")[0].text.strip()
            print(location)
            region = block.select(".fdn-inline-split-list.fdn-teaser-infoline span.uk-text-muted")[0].text.strip()



        if block.select(".fdn-teaser-description"):
            description = block.select(".fdn-teaser-description")[0].text.strip()
            print(description)

        if block.select(".uk-margin-xsmall-top"):
            price = block.select(".uk-margin-xsmall-top")[0].text.strip()
            print(price)
        if any([title, time, venue, location, description]):
            table_data.append([title, time, venue, location, region, description, price])
df = pd.DataFrame(table_data[1:], columns=table_data[0])
df.to_csv("pittscitypaper_events.csv", index=False)

    


Thursday Night Jazz: Yoko Suzuki Spotlights the Music of Toshiko Akiyoshi
Thu., Oct. 17, 7-8:30 p.m.
City of Asylum @ Alphabet City
40 W. North Ave, Pittsburgh
Dr. Yoko Suzuki, an ethnomusicologist and excellent saxophonist, returns to City of Asylum to pay homage to female musicians who were unappreciated in their time.
4124351110
Free
Ritual Sound Movement Healing Art - Wellness Workshop & Quarterly Immersion
Sun., Oct. 20, 10:30 a.m.-3 p.m.
Wightman School
5604 Solway, Pittsburgh
Join us for an interactive, family-friendly day of healing as we uplift our minds, bodies, and hearts through a sampling of wellness practices that are rooted in music, movement, and creative expression. **
Ft: Yoga, Live Music, Dance, Sound Healing, and More! **
Registration for this event is free with a suggested donation at the door.
Everyone who registers will be entered into a wellness raffle giveaway! **
If you would like to support this event but cannot attend in person, please consider donating thro

In [49]:
a = [None, 1]
if any(a):
    print('yes')

yes
